In [1]:
# ===============================================================
# 1) IMPORT LIBRARIES
# ===============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay
import shap
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [2]:
# ===============================================================
# 1) LOAD DATASETS
# ===============================================================

from pathlib import Path
OUT = Path.home() / "ml_outputs"
print(OUT)

data = pd.read_csv(OUT / 'Cleaned_Features_for_ML.csv', index_col=0, parse_dates=True)

C:\Users\dax_a\ml_outputs


In [3]:
# ===============================================================
# STEP — REDUCE DATASET TO TOP 20 FEATURES + REQUIRED COLUMNS
# ===============================================================

# Top 20 selected features
top20_features = [
    "US10Y", "LQD", "HangSeng", "IEF", "DowJones", "BND", "TLT",
    "Nikkei225", "MSCIWorld", "MA20", "Imports_GDP_Pct", "US2Y",
    "Momentum", "S&P500", "Recession_Probability", "Exports_GDP_Pct",
    "DAX", "Inflation_Annual_Pct", "Fed_Funds_Rate", "CAC40"
]


# Columns that MUST remain in the dataset
mandatory_columns = ["Apple", "Return", "Direction", "target_index"]

# Make sure they exist
for col in mandatory_columns:
    if col not in data.columns:
        print(f"Warning: column '{col}' was not found in data.")

# Build the new reduced dataframe
keep_columns = top20_features + mandatory_columns
data = data[keep_columns].copy()

print("New dataset shape:", data.shape)
print("Columns kept:", data.columns.tolist())

New dataset shape: (3970, 24)
Columns kept: ['US10Y', 'LQD', 'HangSeng', 'IEF', 'DowJones', 'BND', 'TLT', 'Nikkei225', 'MSCIWorld', 'MA20', 'Imports_GDP_Pct', 'US2Y', 'Momentum', 'S&P500', 'Recession_Probability', 'Exports_GDP_Pct', 'DAX', 'Inflation_Annual_Pct', 'Fed_Funds_Rate', 'CAC40', 'Apple', 'Return', 'Direction', 'target_index']


In [22]:
print(data.info())
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3970 entries, 2010-07-29 to 2025-11-03
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Recession_Probability    3970 non-null   float64
 1   HangSeng                 3970 non-null   float64
 2   LQD                      3970 non-null   float64
 3   Unemployment_Rate        3970 non-null   float64
 4   OECD_Unemp_rate_pct_USA  3970 non-null   float64
 5   BND                      3970 non-null   float64
 6   Imports_GDP_Pct          3970 non-null   float64
 7   US10Y                    3970 non-null   float64
 8   IEF                      3970 non-null   float64
 9   Nikkei225                3970 non-null   float64
 10  Fed_Funds_Rate           3970 non-null   float64
 11  Inflation_Annual_Pct     3970 non-null   float64
 12  Exports_GDP_Pct          3970 non-null   float64
 13  TLT                      3970 non-null   float64
 14  MA50  

In [9]:
# ===============================================================
# SUPPRESS ARIMA WARNINGS SAFELY
# ===============================================================
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ValueWarning)

# ===============================================================
# ARIMA MODEL
# ===============================================================
from statsmodels.tsa.arima.model import ARIMA

data = data.sort_index()

model = ARIMA(data["Return"], order=(2,1,2))
model_fit = model.fit()

forecast = model_fit.predict(start=1, end=len(data), dynamic=False)
forecast.index = data.index

data["ARIMA_Return_Forecast"] = forecast.shift(1)
data = data.dropna(subset=["ARIMA_Return_Forecast"])


In [10]:
print(data.info())
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3968 entries, 2010-08-02 to 2025-11-03
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   US10Y                  3968 non-null   float64
 1   LQD                    3968 non-null   float64
 2   HangSeng               3968 non-null   float64
 3   IEF                    3968 non-null   float64
 4   DowJones               3968 non-null   float64
 5   BND                    3968 non-null   float64
 6   TLT                    3968 non-null   float64
 7   Nikkei225              3968 non-null   float64
 8   MSCIWorld              3968 non-null   float64
 9   MA20                   3968 non-null   float64
 10  Imports_GDP_Pct        3968 non-null   float64
 11  US2Y                   3968 non-null   float64
 12  Momentum               3968 non-null   float64
 13  S&P500                 3968 non-null   float64
 14  Recession_Probability  3968 non-null  

In [12]:
# ===============================================================
# 0) IMPORTS
# ===============================================================
import numpy as np
import pandas as pd
import warnings

# ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# ARIMA
from statsmodels.tools.sm_exceptions import ValueWarning
from statsmodels.tsa.arima.model import ARIMA

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ValueWarning)

np.random.seed(42)

print("All libraries imported.")


# ===============================================================
# 1) PREPARE DATA
# ===============================================================
data = data.sort_index()

if "Return" not in data.columns:
    raise ValueError("The dataset must contain a column named 'Return'.")

if "Direction" not in data.columns:
    raise ValueError("The dataset must contain a column named 'Direction'.")

print("Dataset loaded & sorted.")


# ===============================================================
# 2) CREATE ARIMA FEATURE
# ===============================================================
print("Fitting ARIMA(2,1,2)...")

arima_model = ARIMA(data["Return"], order=(2, 1, 2))
arima_fit = arima_model.fit()

forecast = arima_fit.predict(start=1, end=len(data), dynamic=False)
forecast.index = data.index
data["ARIMA_Return_Forecast"] = forecast.shift(1)

data = data.dropna(subset=["ARIMA_Return_Forecast"])
print("ARIMA_Return_Forecast feature created.")


# ===============================================================
# 3) CLEAN FULL DATASET (NaN, inf, extreme values)
# ===============================================================
print("Cleaning NaN and infinite values...")

data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(method="ffill", inplace=True)
data.fillna(method="bfill", inplace=True)

# Cap extreme values (finance best practice)
data = data.clip(lower=-1e6, upper=1e6)

print("Cleaning completed.")


# ===============================================================
# 4) FEATURE SELECTION (ANOVA — Top 20)
# ===============================================================
print("Running SelectKBest (ANOVA)...")

# Prepare features
features = data.drop(columns=["Direction"])
target = data["Direction"]

# Standardize features for ANOVA
scaler_fs = StandardScaler()
X_scaled = scaler_fs.fit_transform(features)

selector = SelectKBest(score_func=f_classif, k=20)
selector.fit(X_scaled, target)

ranking_df = pd.DataFrame({
    "Feature": features.columns,
    "ANOVA_F_score": selector.scores_,
    "p_value": selector.pvalues_,
    "Selected": selector.get_support()
}).sort_values(by="ANOVA_F_score", ascending=False)

print("Top 20 features:")
top20_features = ranking_df.head(20)["Feature"].tolist()
print(top20_features)


# ===============================================================
# 5) BUILD ML DATASET
# ===============================================================
X = data[top20_features]
y = data["Direction"]

# Final cleaning before feeding models
imputer = SimpleImputer(strategy="median")
X = imputer.fit_transform(X)

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split (time series: no shuffling)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print("Train/Test dataset ready.")


# ===============================================================
# 6) MODEL SET
# ===============================================================
models = [
    ("LogReg", LogisticRegression(max_iter=500)),
    ("RandomForest", RandomForestClassifier(n_estimators=300)),
    ("ExtraTrees", ExtraTreesClassifier(n_estimators=300)),
    ("GradientBoost", GradientBoostingClassifier()),
    ("AdaBoost", AdaBoostClassifier()),
    ("SVM", SVC(probability=True)),
    ("KNN", KNeighborsClassifier(n_neighbors=7)),
    ("MLP", MLPClassifier(max_iter=500))
]

print("Models initialized.")


# ===============================================================
# 7) TRAIN & EVALUATE
# ===============================================================
results = []

for name, model in models:
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    results.append((name, acc, f1, auc))

    print(f"{name} — ACC: {acc:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")


# ===============================================================
# 8) RESULTS TABLE
# ===============================================================
results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "F1", "ROC-AUC"])
print("\nFinal Results:")
display(results_df.sort_values(by="ROC-AUC", ascending=False))


All libraries imported.
Dataset loaded & sorted.
Fitting ARIMA(2,1,2)...
ARIMA_Return_Forecast feature created.
Cleaning NaN and infinite values...
Cleaning completed.
Running SelectKBest (ANOVA)...
Top 20 features:
['US10Y', 'LQD', 'HangSeng', 'IEF', 'DowJones', 'BND', 'TLT', 'Nikkei225', 'MSCIWorld', 'MA20', 'Imports_GDP_Pct', 'ARIMA_Return_Forecast', 'US2Y', 'Momentum', 'S&P500', 'Recession_Probability', 'Exports_GDP_Pct', 'DAX', 'Inflation_Annual_Pct', 'CAC40']
Train/Test dataset ready.
Models initialized.

Training LogReg...
LogReg — ACC: 0.5202, F1: 0.5809, AUC: 0.5143

Training RandomForest...
RandomForest — ACC: 0.5050, F1: 0.5549, AUC: 0.5031

Training ExtraTrees...
ExtraTrees — ACC: 0.5038, F1: 0.5651, AUC: 0.4934

Training GradientBoost...
GradientBoost — ACC: 0.5076, F1: 0.5295, AUC: 0.5016

Training AdaBoost...
AdaBoost — ACC: 0.5416, F1: 0.5583, AUC: 0.5297

Training SVM...
SVM — ACC: 0.5151, F1: 0.6214, AUC: 0.4625

Training KNN...
KNN — ACC: 0.4748, F1: 0.5341, AUC: 0.4

,Model,Accuracy,F1,ROC-AUC
4,AdaBoost,0.541562,0.558252,0.529709
0,LogReg,0.520151,0.580858,0.514328
1,RandomForest,0.505038,0.554926,0.503085
3,GradientBoost,0.507557,0.529483,0.501638
2,ExtraTrees,0.503778,0.565121,0.493448
7,MLP,0.491184,0.561822,0.483878
6,KNN,0.474811,0.534078,0.469175
5,SVM,0.515113,0.621436,0.462480


In [13]:
# ===============================================================
# ADVANCED MODELS – ENSEMBLE & META-LEARNING COMPARISON
# ===============================================================
print("\n====================================================")
print("   ADVANCED ML MODELS – ENSEMBLE & STACKING")
print("====================================================\n")

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    BaggingClassifier,
    VotingClassifier,
    StackingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


# ===============================================================
# PREPROCESSING PIPELINE (Imputation + Scaling)
# ===============================================================
def make_pipeline(model):
    return Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("model", model)
    ])


# ---------------------------------------------------------------
# 1) BASE MODELS (for consistency)
# ---------------------------------------------------------------
base_svm = SVC(kernel='rbf', probability=True, random_state=0)
base_dt = DecisionTreeClassifier(random_state=0)


# ===============================================================
# 2) TUNED MODELS WITH GRID SEARCH (with Pipeline)
# ===============================================================
param_svm = {
    "model__C": [0.1, 1, 10],
    "model__gamma": ["scale", 0.01, 0.001]
}

tuned_svm = GridSearchCV(
    make_pipeline(SVC(probability=True, random_state=0)),
    param_svm,
    cv=5,
    n_jobs=-1,
    error_score="raise"
)
tuned_svm.fit(X_train, y_train)


param_dt = {
    "model__max_depth": [3, 5, 7, None],
    "model__min_samples_split": [2, 5, 10]
}

tuned_dt = GridSearchCV(
    make_pipeline(DecisionTreeClassifier(random_state=0)),
    param_dt,
    cv=5,
    n_jobs=-1,
    error_score="raise"
)
tuned_dt.fit(X_train, y_train)


# ===============================================================
# 3) BAGGING MODELS (corrected for scikit-learn ≥ 1.4)
# ===============================================================

bagging_svm = make_pipeline(
    BaggingClassifier(
        estimator=SVC(kernel='rbf', probability=True),
        n_estimators=20,
        max_samples=0.8,
        random_state=0
    )
)

bagging_dt = make_pipeline(
    BaggingClassifier(
        estimator=DecisionTreeClassifier(),
        n_estimators=50,
        max_samples=0.8,
        random_state=0
    )
)


# ===============================================================
# 4) VOTING CLASSIFIER (pipelined)
# ===============================================================
voting = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", VotingClassifier(
        estimators=[
            ('svm', SVC(kernel='rbf', probability=True)),
            ('dt', DecisionTreeClassifier())
        ],
        voting='soft'
    ))
])


# ===============================================================
# 5) STACKING CLASSIFIER (with corrected Bagging)
# ===============================================================
stacking = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", StackingClassifier(
        estimators=[
            ('svm', SVC(kernel='rbf', probability=True)),
            ('dt', DecisionTreeClassifier()),
            ('bag_svm', BaggingClassifier(
                estimator=SVC(kernel='rbf', probability=True),
                n_estimators=10)),
            ('bag_dt', BaggingClassifier(
                estimator=DecisionTreeClassifier(),
                n_estimators=10))
        ],
        final_estimator=LogisticRegression(max_iter=500),
        passthrough=True,
        n_jobs=-1
    ))
])


# ===============================================================
# TRAIN + EVALUATE ALL MODELS
# ===============================================================
models = {
    "Tuned SVM": tuned_svm.best_estimator_,
    "Tuned Decision Tree": tuned_dt.best_estimator_,
    "Bagging SVM": bagging_svm,
    "Bagging Decision Tree": bagging_dt,
    "Voting (SVM + DT)": voting,
    "Stacking Meta-Model": stacking
}

results = []

for name, model in models.items():
    print(f"\n▶ Training {name} ...")
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    results.append([name, acc, f1, auc])

    print(f"   Accuracy: {acc:.4f}")
    print(f"   F1-score: {f1:.4f}")
    print(f"   AUC: {auc:.4f}")


# ===============================================================
# SUMMARY TABLE
# ===============================================================
df_results = pd.DataFrame(results, columns=["Model", "Accuracy", "F1-score", "AUC"])
df_results.sort_values(by="F1-score", ascending=False)


   ADVANCED ML MODELS – ENSEMBLE & STACKING


▶ Training Tuned SVM ...
   Accuracy: 0.5390
   F1-score: 0.6914
   AUC: 0.4768

▶ Training Tuned Decision Tree ...
   Accuracy: 0.4761
   F1-score: 0.4541
   AUC: 0.4790

▶ Training Bagging SVM ...
   Accuracy: 0.5252
   F1-score: 0.6887
   AUC: 0.4826

▶ Training Bagging Decision Tree ...
   Accuracy: 0.5139
   F1-score: 0.5246
   AUC: 0.5100

▶ Training Voting (SVM + DT) ...
   Accuracy: 0.5063
   F1-score: 0.5075
   AUC: 0.4932

▶ Training Stacking Meta-Model ...
   Accuracy: 0.5139
   F1-score: 0.5768
   AUC: 0.5117


,Model,Accuracy,F1-score,AUC
0,Tuned SVM,0.539043,0.691400,0.476779
2,Bagging SVM,0.525189,0.688687,0.482619
5,Stacking Meta-Model,0.513854,0.576754,0.511688
3,Bagging Decision Tree,0.513854,0.524631,0.509974
4,Voting (SVM + DT),0.506297,0.507538,0.493213
1,Tuned Decision Tree,0.476071,0.454068,0.479047


In [14]:
# ===============================================================
# 0) IMPORTS
# ===============================================================
import numpy as np
import pandas as pd

from statsmodels.tsa.arima.model import ARIMA

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import (
    AdaBoostClassifier, GradientBoostingClassifier,
    RandomForestClassifier, ExtraTreesClassifier
)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# ===============================================================
# 1) SORT DATA & CHECK COLUMNS
# ===============================================================
# Make sure data is sorted in time
data = data.sort_index()

required_cols = ["Return", "Direction"]
for c in required_cols:
    if c not in data.columns:
        raise ValueError(f"Missing required column: {c}")

# ===============================================================
# 2) BUILD ARIMA FORECAST FEATURE
# ===============================================================
# ARIMA on 'Return' – order (2,1,2) as we discussed
arima_model = ARIMA(data["Return"], order=(2, 1, 2))
arima_fit = arima_model.fit()

print(arima_fit.summary())  # optional

# In-sample one-step-ahead predictions
arima_pred = arima_fit.predict(start=1, end=len(data), dynamic=False)

# Align index with data
arima_pred.index = data.index

# Shift by 1: forecast at t is based only on information up to t-1
data["ARIMA_Return_Forecast"] = arima_pred.shift(1)

# You can also create a directional signal from ARIMA, optional:
data["ARIMA_Direction"] = (data["ARIMA_Return_Forecast"] > 0).astype(int)

# Because of the shift, first row is NaN → drop it (or more if needed)
data_model = data.dropna(subset=["ARIMA_Return_Forecast"]).copy()

print("Data shape after adding ARIMA feature and dropping NaNs:", data_model.shape)

# ===============================================================
# 3) DEFINE FEATURES & TARGET
# ===============================================================
# Drop raw price & return & target, keep everything else including ARIMA feature
X = data_model.drop(columns=["Apple", "Return", "Direction"])
y = data_model["Direction"]

print("Final feature columns used:")
print(X.columns.tolist())

# ===============================================================
# 4) TRAIN / TEST SPLIT (TIME-SERIES FRIENDLY)
# ===============================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print("Train shape:", X_train.shape, " Test shape:", X_test.shape)

# ===============================================================
# 5) DEFINE CLASSIFICATION MODELS
# ===============================================================
models = [
    ('LR',  LogisticRegression(max_iter=5000, random_state=RANDOM_STATE)),
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('CART', DecisionTreeClassifier(max_depth=6, random_state=RANDOM_STATE)),
    ('SVC', SVC(kernel='rbf', probability=True, random_state=RANDOM_STATE)),
    ('MLP', MLPClassifier(hidden_layer_sizes=(64, 32),
                          max_iter=5000, random_state=RANDOM_STATE)),
    # Boosting
    ('ABR', AdaBoostClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('GBR', GradientBoostingClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    # Bagging
    ('RFR', RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('ETR', ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE))
]

# ===============================================================
# 6) EVALUATE MODELS WITH ARIMA FEATURE
# ===============================================================
results = []

print("\nRunning classification models with ARIMA feature...\n")

for name, model in models:
    # Fit the model
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    results.append([name, acc, f1, auc])

    print(f"{name}: "
          f"Accuracy={acc:.4f} | F1={f1:.4f} | ROC-AUC={auc:.4f}")

# ===============================================================
# 7) RESULTS TABLE
# ===============================================================
results_df = pd.DataFrame(
    results, columns=["Model", "Accuracy", "F1", "ROC_AUC"]
).sort_values(by="ROC_AUC", ascending=False)

print("\n=========== Ranked Results (by ROC-AUC) ===========")
print(results_df.to_string(index=False))


                               SARIMAX Results                                
Dep. Variable:                 Return   No. Observations:                 3967
Model:                 ARIMA(2, 1, 2)   Log Likelihood               10423.761
Date:                Wed, 19 Nov 2025   AIC                         -20837.522
Time:                        09:18:55   BIC                         -20806.094
Sample:                             0   HQIC                        -20826.377
                               - 3967                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.9839      0.060    -16.380      0.000      -1.102      -0.866
ar.L2         -0.0397      0.010     -3.983      0.000      -0.059      -0.020
ma.L1         -0.0494      0.059     -0.840      0.4

In [15]:
# ===============================================================
# 0) IMPORTS
# ===============================================================
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# ML Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import (
    AdaBoostClassifier, GradientBoostingClassifier,
    RandomForestClassifier, ExtraTreesClassifier
)

# Deep Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# ===============================================================
# 1) SORT DATA
# ===============================================================
data = data.sort_index()

# ===============================================================
# 2) BUILD SUPERVISED SEQUENCE DATA FOR LSTM
#    Here we use past 20 returns to predict return(t+1)
# ===============================================================
SEQ_LEN = 20

returns = data["Return"].values.reshape(-1, 1)

X_seq = []
y_seq = []

for i in range(SEQ_LEN, len(returns)):
    X_seq.append(returns[i-SEQ_LEN:i])
    y_seq.append(returns[i])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

print("LSTM input shape:", X_seq.shape)

# ===============================================================
# 3) TRAIN/TEST SPLIT (no shuffle)
# ===============================================================
split = int(0.8 * len(X_seq))

X_train_seq = X_seq[:split]
X_test_seq  = X_seq[split:]

y_train_seq = y_seq[:split]
y_test_seq  = y_seq[split:]

# ===============================================================
# 4) DEFINE LSTM MODEL
# ===============================================================
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LEN, 1)),
    LSTM(32),
    Dense(1)
])

model.compile(loss="mse", optimizer=Adam(learning_rate=0.001))

es = EarlyStopping(patience=10, restore_best_weights=True)

# ===============================================================
# 5) TRAIN LSTM
# ===============================================================
history = model.fit(
    X_train_seq, y_train_seq,
    validation_split=0.1,
    epochs=50,
    batch_size=32,
    callbacks=[es],
    verbose=1
)

# ===============================================================
# 6) FORECAST RETURNS WITH LSTM
# ===============================================================
lstm_forecast = model.predict(X_seq).flatten()

# Shift by 1 to avoid lookahead bias
lstm_forecast = pd.Series(lstm_forecast, index=data.index[SEQ_LEN:])
lstm_forecast = lstm_forecast.shift(1)

data["LSTM_Return_Forecast"] = lstm_forecast

# Optional: binary directional prediction from LSTM
data["LSTM_Direction"] = (data["LSTM_Return_Forecast"] > 0).astype(int)

# Drop the initial NaNs created by LSTM sequence length
data_ml = data.dropna(subset=["LSTM_Return_Forecast"])

print("Data shape after adding LSTM feature:", data_ml.shape)

# ===============================================================
# 7) PREPARE ML CLASSIFICATION DATA
# ===============================================================
X = data_ml.drop(columns=["Apple", "Return", "Direction"])
y = data_ml["Direction"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# ===============================================================
# 8) DEFINE ML MODELS
# ===============================================================
models = [
    ('LR',  LogisticRegression(max_iter=5000, random_state=RANDOM_STATE)),
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('CART', DecisionTreeClassifier(max_depth=6, random_state=RANDOM_STATE)),
    ('SVC', SVC(kernel='rbf', probability=True, random_state=RANDOM_STATE)),
    ('MLP', MLPClassifier(hidden_layer_sizes=(64, 32),
                          max_iter=5000, random_state=RANDOM_STATE)),
    ('ABR', AdaBoostClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('GBR', GradientBoostingClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('RFR', RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('ETR', ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE))
]

# ===============================================================
# 9) EVALUATE THE ML CLASSIFIERS (WITH LSTM FEATURE)
# ===============================================================
results = []

print("\nRunning ML classifiers WITH LSTM feature...\n")

for name, model_ml in models:
    model_ml.fit(X_train, y_train)

    y_pred = model_ml.predict(X_test)
    y_proba = model_ml.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    results.append([name, acc, f1, auc])

    print(f"{name}: Accuracy={acc:.4f} | F1={f1:.4f} | ROC-AUC={auc:.4f}")

# ===============================================================
# 10) RESULTS TABLE
# ===============================================================
results_df = pd.DataFrame(
    results, columns=["Model", "Accuracy", "F1", "ROC_AUC"]
).sort_values(by="ROC_AUC", ascending=False)

print("\n=========== Ranked Results (LSTM-enhanced ML) ===========")
print(results_df.to_string(index=False))


LSTM input shape: (3947, 20, 1)
Epoch 1/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 3.1124e-04 - val_loss: 3.6382e-04
Epoch 2/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.0916e-04 - val_loss: 3.8546e-04
Epoch 3/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3.0670e-04 - val_loss: 3.7087e-04
Epoch 4/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.0614e-04 - val_loss: 3.6176e-04
Epoch 5/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.0874e-04 - val_loss: 3.6164e-04
Epoch 6/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.0698e-04 - val_loss: 3.6220e-04
Epoch 7/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3.0538e-04 - val_loss: 3.6222e-04
Epoch 8/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.0567e-04 - val_loss: 3.6252e-04
Epoch 9/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3.0750e-04 - val_loss: 3.6138e-04
Epoch 10/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.0646e-04 - val_loss: 3.8014e-04
Epoch 11/50
89/89 ━━━━━━━━━━━━━━━━━━━

In [16]:
# ===============================================================
# 0) IMPORTS
# ===============================================================
import numpy as np
import pandas as pd

from statsmodels.tsa.arima.model import ARIMA

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import (
    AdaBoostClassifier, GradientBoostingClassifier,
    RandomForestClassifier, ExtraTreesClassifier
)

# Deep learning (LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# ===============================================================
# 1) DATA PREP: SORT + BASIC CHECKS
# ===============================================================
# Assume 'data' already exists with 'Return' and 'Direction'
data = data.sort_index()

if "Return" not in data.columns or "Direction" not in data.columns:
    raise ValueError("Data must contain 'Return' and 'Direction' columns.")

# ===============================================================
# 2) ARIMA FORECAST: ARIMA_Return_Forecast
# ===============================================================
print("Fitting ARIMA(2,1,2) on returns...")

arima_model = ARIMA(data["Return"], order=(2, 1, 2))
arima_fit = arima_model.fit()

arima_pred = arima_fit.predict(start=1, end=len(data), dynamic=False)
arima_pred.index = data.index
data["ARIMA_Return_Forecast"] = arima_pred.shift(1)  # avoid look-ahead

# Optional directional signal from ARIMA
data["ARIMA_Direction"] = (data["ARIMA_Return_Forecast"] > 0).astype(int)

# ===============================================================
# 3) LSTM FORECAST: LSTM_Return_Forecast
# ===============================================================
print("Building LSTM sequences...")

SEQ_LEN = 20
returns = data["Return"].values.reshape(-1, 1)

X_seq, y_seq = [], []
for i in range(SEQ_LEN, len(returns)):
    X_seq.append(returns[i-SEQ_LEN:i])
    y_seq.append(returns[i])
X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

print("LSTM input shape:", X_seq.shape)

# Split for LSTM (purely for forecasting, still time-ordered)
split_idx = int(0.8 * len(X_seq))
X_train_seq, X_test_seq = X_seq[:split_idx], X_seq[split_idx:]
y_train_seq, y_test_seq = y_seq[:split_idx], y_seq[split_idx:]

# Define LSTM
model_lstm = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LEN, 1)),
    LSTM(32),
    Dense(1)
])
model_lstm.compile(loss="mse", optimizer=Adam(learning_rate=0.001))

es = EarlyStopping(patience=10, restore_best_weights=True)

print("Training LSTM...")
history = model_lstm.fit(
    X_train_seq, y_train_seq,
    validation_split=0.1,
    epochs=50,
    batch_size=32,
    callbacks=[es],
    verbose=1
)

print("Generating LSTM forecasts...")
lstm_forecast = model_lstm.predict(X_seq).flatten()

# Align with dates: first forecast corresponds to index[SEQ_LEN:]
lstm_series = pd.Series(lstm_forecast, index=data.index[SEQ_LEN:])
# Shift by 1 to avoid look-ahead
lstm_series = lstm_series.shift(1)

data["LSTM_Return_Forecast"] = lstm_series
data["LSTM_Direction"] = (data["LSTM_Return_Forecast"] > 0).astype(int)

# ===============================================================
# 4) DROP NaNs CREATED BY ARIMA/LSTM
# ===============================================================
data_ml = data.dropna(subset=["ARIMA_Return_Forecast", "LSTM_Return_Forecast"]).copy()
data_ml = data_ml.sort_index()

print("Data shape after ARIMA+LSTM features:", data_ml.shape)

# ===============================================================
# 5) FEATURES & TARGET FOR ML
# ===============================================================
X_full = data_ml.drop(columns=["Apple", "Return", "Direction"])
y_full = data_ml["Direction"]

# Time-safe train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.2, shuffle=False
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# Keep ARIMA/LSTM features separately for meta-layer
arima_train = X_train["ARIMA_Return_Forecast"].values.reshape(-1, 1)
lstm_train  = X_train["LSTM_Return_Forecast"].values.reshape(-1, 1)

arima_test = X_test["ARIMA_Return_Forecast"].values.reshape(-1, 1)
lstm_test  = X_test["LSTM_Return_Forecast"].values.reshape(-1, 1)

# Base ML features without ARIMA/LSTM if you want to avoid duplicating them
base_feature_cols = [c for c in X_train.columns
                     if c not in ["ARIMA_Return_Forecast", "LSTM_Return_Forecast"]]
X_train_base = X_train[base_feature_cols]
X_test_base  = X_test[base_feature_cols]

print("Base feature count (excluding ARIMA/LSTM):", len(base_feature_cols))

# ===============================================================
# 6) DEFINE BASE CLASSIFICATION MODELS
# ===============================================================
base_models = [
    ('LR',  LogisticRegression(max_iter=5000, random_state=RANDOM_STATE)),
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('CART', DecisionTreeClassifier(max_depth=6, random_state=RANDOM_STATE)),
    ('SVC', SVC(kernel='rbf', probability=True, random_state=RANDOM_STATE)),
    ('MLP', MLPClassifier(hidden_layer_sizes=(64, 32),
                          max_iter=5000, random_state=RANDOM_STATE)),
    ('ABR', AdaBoostClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('GBR', GradientBoostingClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('RFR', RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('ETR', ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE))
]

# ===============================================================
# 7) STACKING – BUILD OUT-OF-FOLD META-FEATURES (TRAIN SET)
# ===============================================================
print("\nBuilding out-of-fold meta-features (stacking)...")

tscv = TimeSeriesSplit(n_splits=5)
n_train = X_train_base.shape[0]
n_models = len(base_models)

# Only base models here: shape (n_train, n_models)
meta_train_base = np.full((n_train, n_models), np.nan)

for m_idx, (name, model) in enumerate(base_models):
    print(f"  OOF for base model: {name}")

    oof_pred = np.full(n_train, np.nan)

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train_base)):
        X_tr, X_val = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        proba_val = model.predict_proba(X_val)[:, 1]
        oof_pred[val_idx] = proba_val

    # Now fill missing OOF predictions (earlier samples never validated)
    if np.isnan(oof_pred).any():
        model.fit(X_train_base, y_train)
        full_pred = model.predict_proba(X_train_base)[:, 1]
        oof_pred = np.where(np.isnan(oof_pred), full_pred, oof_pred)

    meta_train_base[:, m_idx] = oof_pred

# Combine with ARIMA and LSTM features
meta_features_train = np.concatenate(
    [meta_train_base, arima_train, lstm_train], axis=1
)

print("Final meta_features_train shape:", meta_features_train.shape)  # MUST be (N, 11)

# ===============================================================
# 8) TRAIN META MODEL
# ===============================================================
meta_clf = LogisticRegression(max_iter=5000, random_state=RANDOM_STATE)
meta_clf.fit(meta_features_train, y_train)

# ===============================================================
# 9) BUILD META-FEATURES FOR TEST SET
# ===============================================================
print("\nBuilding meta-features for TEST set...")

meta_test_base = np.zeros((X_test_base.shape[0], n_models))

for m_idx, (name, model) in enumerate(base_models):
    print(f"  Train full + predict test for: {name}")
    model.fit(X_train_base, y_train)
    meta_test_base[:, m_idx] = model.predict_proba(X_test_base)[:, 1]

# Add ARIMA + LSTM
meta_features_test = np.concatenate(
    [meta_test_base, arima_test, lstm_test], axis=1
)

print("Final meta_features_test shape:", meta_features_test.shape)  # MUST be (N_test, 11)


# ===============================================================
# 10) EVALUATE META-MODEL
# ===============================================================
y_pred_meta = meta_clf.predict(meta_features_test)
y_proba_meta = meta_clf.predict_proba(meta_features_test)[:, 1]

acc_meta = accuracy_score(y_test, y_pred_meta)
f1_meta  = f1_score(y_test, y_pred_meta)
auc_meta = roc_auc_score(y_test, y_proba_meta)

print("\n=========== META-MODEL PERFORMANCE (ARIMA + LSTM + ML) ===========")
print(f"Accuracy = {acc_meta:.4f}")
print(f"F1       = {f1_meta:.4f}")
print(f"ROC-AUC  = {auc_meta:.4f}")


Fitting ARIMA(2,1,2) on returns...
Building LSTM sequences...
LSTM input shape: (3947, 20, 1)
Training LSTM...
Epoch 1/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 3.1823e-04 - val_loss: 3.6345e-04
Epoch 2/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.1153e-04 - val_loss: 3.6941e-04
Epoch 3/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.0868e-04 - val_loss: 3.6249e-04
Epoch 4/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 3.1008e-04 - val_loss: 3.6473e-04
Epoch 5/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.0577e-04 - val_loss: 3.6076e-04
Epoch 6/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.0680e-04 - val_loss: 3.6114e-04
Epoch 7/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3.0655e-04 - val_loss: 3.6250e-04
Epoch 8/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.0601e-04 - val_loss: 3.6110e-04
Epoch 9/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.0541e-04 - val_loss: 3.6271e-04
Epoch 10/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step 

In [17]:
# ===============================================================
# 0) IMPORTS
# ===============================================================
import numpy as np
import pandas as pd

from statsmodels.tsa.arima.model import ARIMA

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report
)

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import (
    AdaBoostClassifier, GradientBoostingClassifier,
    RandomForestClassifier, ExtraTreesClassifier
)

# Deep learning (LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# ===============================================================
# 1) PREP DATA + CREATE 8-DAY FORWARD TREND TARGET
# ===============================================================
data = data.sort_index()

required_cols = ["Apple", "Return", "Direction"]
for c in required_cols:
    if c not in data.columns:
        raise ValueError(f"Missing column: {c}")

# Forward-looking 8-day average of daily Direction
# Direction_Forward8(t) = mean(Direction(t+1..t+8))
data["Direction_Forward8"] = (
    data["Direction"]
    .rolling(8)
    .mean()
    .shift(-8)
)

# Binary trend label:
#  1 if > 60% of future 8 days are up
#  0 if < 40% of future 8 days are up
#  NaN in between (neutral, we drop them for now)
data["Trend_8D"] = np.where(
    data["Direction_Forward8"] >= 0.6, 1,
    np.where(data["Direction_Forward8"] <= 0.4, 0, np.nan)
)

# Drop rows where target is NaN (start and end)
data = data.dropna(subset=["Trend_8D"]).copy()
data["Trend_8D"] = data["Trend_8D"].astype(int)

print("Data shape after building Trend_8D target:", data.shape)

# ===============================================================
# 2) ARIMA FORECAST FEATURE (ON RETURN)
# ===============================================================
print("Fitting ARIMA(2,1,2) on Return...")

arima_model = ARIMA(data["Return"], order=(2, 1, 2))
arima_fit = arima_model.fit()

arima_pred = arima_fit.predict(start=1, end=len(data), dynamic=False)
arima_pred.index = data.index

# Shift one step to avoid look-ahead
data["ARIMA_Return_Forecast"] = arima_pred.shift(1)
data["ARIMA_Direction"] = (data["ARIMA_Return_Forecast"] > 0).astype(int)

# ===============================================================
# 3) LSTM FORECAST FEATURE (ON RETURN)
# ===============================================================
print("Preparing LSTM sequences...")

SEQ_LEN = 20
returns_arr = data["Return"].values.reshape(-1, 1)

X_seq, y_seq = [], []
for i in range(SEQ_LEN, len(returns_arr)):
    X_seq.append(returns_arr[i-SEQ_LEN:i])
    y_seq.append(returns_arr[i])
X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

print("LSTM input shape:", X_seq.shape)

split_idx = int(0.8 * len(X_seq))
X_train_seq, X_test_seq = X_seq[:split_idx], X_seq[split_idx:]
y_train_seq, y_test_seq = y_seq[:split_idx], y_seq[split_idx:]

model_lstm = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LEN, 1)),
    LSTM(32),
    Dense(1)
])
model_lstm.compile(loss="mse", optimizer=Adam(learning_rate=0.001))

es = EarlyStopping(patience=10, restore_best_weights=True)

print("Training LSTM...")
model_lstm.fit(
    X_train_seq, y_train_seq,
    validation_split=0.1,
    epochs=50,
    batch_size=32,
    callbacks=[es],
    verbose=1
)

print("Forecasting with LSTM...")
lstm_pred = model_lstm.predict(X_seq).flatten()
lstm_series = pd.Series(lstm_pred, index=data.index[SEQ_LEN:])

# Shift by 1 to avoid look-ahead
data["LSTM_Return_Forecast"] = lstm_series.shift(1)
data["LSTM_Direction"] = (data["LSTM_Return_Forecast"] > 0).astype(int)

# ===============================================================
# 4) DROP NaNs (from ARIMA/LSTM shifts and sequence start)
# ===============================================================
data_ml = data.dropna(subset=["ARIMA_Return_Forecast", "LSTM_Return_Forecast"]).copy()
print("Data shape after ARIMA+LSTM dropna:", data_ml.shape)

# ===============================================================
# 5) FEATURES & TARGET FOR STACKING (Option C)
# ===============================================================
# Use ALL available predictors except:
#  - raw price/return
#  - daily Direction
#  - forward-label helpers
X_full = data_ml.drop(columns=[
    "Apple", "Return", "Direction", "Direction_Forward8"
])
y_full = data_ml["Trend_8D"]

X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.2, shuffle=False
)

# Keep ARIMA/LSTM forecasts separate for meta-layer
arima_train = X_train["ARIMA_Return_Forecast"].values.reshape(-1, 1)
lstm_train  = X_train["LSTM_Return_Forecast"].values.reshape(-1, 1)
arima_test  = X_test["ARIMA_Return_Forecast"].values.reshape(-1, 1)
lstm_test   = X_test["LSTM_Return_Forecast"].values.reshape(-1, 1)

# Base feature set excludes ARIMA/LSTM forecasts
base_feature_cols = [
    c for c in X_train.columns
    if c not in ["ARIMA_Return_Forecast", "LSTM_Return_Forecast"]
]
X_train_base = X_train[base_feature_cols]
X_test_base  = X_test[base_feature_cols]

print("Base feature count (excluding ARIMA/LSTM):", len(base_feature_cols))

# ===============================================================
# 6) DEFINE BASE CLASSIFICATION MODELS
# ===============================================================
base_models = [
    ('LR',  LogisticRegression(max_iter=5000, random_state=RANDOM_STATE)),
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('CART', DecisionTreeClassifier(max_depth=6, random_state=RANDOM_STATE)),
    ('SVC',  SVC(kernel='rbf', probability=True, random_state=RANDOM_STATE)),
    ('MLP',  MLPClassifier(hidden_layer_sizes=(64, 32),
                           max_iter=5000, random_state=RANDOM_STATE)),
    ('ABR',  AdaBoostClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('GBR',  GradientBoostingClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('RFR',  RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE)),
    ('ETR',  ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE))
]

# ===============================================================
# 7) STACKING – BUILD OUT-OF-FOLD META-FEATURES (TRAIN)
# ===============================================================
print("\nBuilding out-of-fold meta-features for Trend_8D...\n")

tscv = TimeSeriesSplit(n_splits=5)
n_train = X_train_base.shape[0]
n_models = len(base_models)

meta_train_base = np.full((n_train, n_models), np.nan)

for m_idx, (name, model) in enumerate(base_models):
    print(f"  {name} OOF predictions...")
    oof_pred = np.full(n_train, np.nan)

    for fold, (tr_idx, val_idx) in enumerate(tscv.split(X_train_base)):
        X_tr, X_val = X_train_base.iloc[tr_idx], X_train_base.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        oof_pred[val_idx] = model.predict_proba(X_val)[:, 1]

    # Fill NaNs (early samples not in any validation fold)
    if np.isnan(oof_pred).any():
        model.fit(X_train_base, y_train)
        full_pred = model.predict_proba(X_train_base)[:, 1]
        oof_pred = np.where(np.isnan(oof_pred), full_pred, oof_pred)

    meta_train_base[:, m_idx] = oof_pred

# Append ARIMA & LSTM forecasts to meta-features
meta_features_train = np.concatenate(
    [meta_train_base, arima_train, lstm_train],
    axis=1
)

print("meta_features_train shape:", meta_features_train.shape)

# ===============================================================
# 8) TRAIN META-CLASSIFIER (LOGISTIC REGRESSION)
# ===============================================================
meta_clf = LogisticRegression(max_iter=5000, random_state=RANDOM_STATE)
meta_clf.fit(meta_features_train, y_train)

# ===============================================================
# 9) META-FEATURES FOR TEST
# ===============================================================
print("\nBuilding meta-features for TEST...\n")

meta_test_base = np.zeros((X_test_base.shape[0], n_models))

for m_idx, (name, model) in enumerate(base_models):
    print(f"  {name} full-train → test prediction...")
    model.fit(X_train_base, y_train)
    meta_test_base[:, m_idx] = model.predict_proba(X_test_base)[:, 1]

meta_features_test = np.concatenate(
    [meta_test_base, arima_test, lstm_test],
    axis=1
)

print("meta_features_test shape:", meta_features_test.shape)

# ===============================================================
# 10) EVALUATE META-MODEL ON Trend_8D (NO SMOOTHING)
# ===============================================================
y_proba_meta = meta_clf.predict_proba(meta_features_test)[:, 1]
y_pred_meta  = (y_proba_meta >= 0.5).astype(int)

print("\n=========== META-MODEL PERFORMANCE on Trend_8D (raw) ===========")
print(f"Accuracy = {accuracy_score(y_test, y_pred_meta):.4f}")
print(f"F1       = {f1_score(y_test, y_pred_meta):.4f}")
print(f"ROC-AUC  = {roc_auc_score(y_test, y_proba_meta):.4f}")

print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred_meta))

print("\nClassification report:")
print(classification_report(y_test, y_pred_meta, digits=4))

# ===============================================================
# 11) SMOOTHED PREDICTION: USE LAST 8 DAYS MODEL PROBABILITIES
#     forecast at T based on model probs from T-8..T-1
# ===============================================================
proba_series = pd.Series(y_proba_meta, index=X_test.index)

# Rolling mean over past 8 predictions, aligned so that
# proba_smooth(T) uses predictions from T-8..T-1
proba_smooth = proba_series.rolling(window=8).mean().shift(1)

y_pred_smooth = (proba_smooth >= 0.5).astype(int)

# Align with y_test (drop NaNs from smoothing)
valid_idx = proba_smooth.dropna().index
y_test_smooth = y_test.loc[valid_idx]
y_pred_smooth_valid = y_pred_smooth.loc[valid_idx]

print("\n=========== META-MODEL PERFORMANCE with 8-day SMOOTHED forecast ===========")
print(f"Accuracy = {accuracy_score(y_test_smooth, y_pred_smooth_valid):.4f}")
print(f"F1       = {f1_score(y_test_smooth, y_pred_smooth_valid):.4f}")
print(f"ROC-AUC  = {roc_auc_score(y_test_smooth, proba_smooth.loc[valid_idx]):.4f}")

print("\nConfusion matrix (smoothed):")
print(confusion_matrix(y_test_smooth, y_pred_smooth_valid))

print("\nClassification report (smoothed):")
print(classification_report(y_test_smooth, y_pred_smooth_valid, digits=4))


Data shape after building Trend_8D target: (2891, 30)
Fitting ARIMA(2,1,2) on Return...
Preparing LSTM sequences...
LSTM input shape: (2871, 20, 1)
Training LSTM...
Epoch 1/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 3.3400e-04 - val_loss: 4.5569e-04
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.1933e-04 - val_loss: 4.4459e-04
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.2225e-04 - val_loss: 4.4249e-04
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.1625e-04 - val_loss: 4.8252e-04
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.1859e-04 - val_loss: 4.4331e-04
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.1502e-04 - val_loss: 4.5110e-04
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.1595e-04 - val_loss: 4.4388e-04
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.1505e-04 - val_loss: 4.4358e-04
Epoch 9/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.1594e-04 - val_loss: 4.4342e-0